# Modelling and analyzing a simple unbalanced network with Roseau Load Flow solver

### Introduction

This tutorial is meant to introduce the procedure for modelling components of a given low voltage
(LV) network using the _Roseau Load Flow_ solver. It will also cover the process of running a power
flow, accessing results, and analysis such as voltage regulation and energy losses.

Consider the simple LV network shown in the figure below containing a MV/LV, $\Delta$-Y transformer
(20kV/0.4kV, 250 kVA) between the source bus and bus A, a 240 mm², 3-phase line connecting buses A
and B, and three 16 mm² single-phase lines connecting bus B with buses C, D and E each of which
serves as a connection point for a house.

<center> <img style="float: middle;" 
          src="../images/LV_Network_With_Neutral.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network</center>**


### Import the required modules and functions

First we need to import the modules and functions needed for this tutorial. The purpose of the
following code are explained as follows:

- `import roseau.load_flow as rlf `: imports the _Roseau Load Flow_ (RLF) package. This package is
  needed to model the various components of the network such as buses, lines, transformers, etc.

- `import numpy as np`: imports the _numpy_ package which will be used for mathematical manipulations
  including working with arrays, calculating exponents, and handling constants like pi.


In [ ]:
import numpy as np
import roseau.load_flow as rlf

### Creating the buses

After importing all the necessary modules, we can move on to the modelling of the given LV network.
We start with the buses which are the nodes of the network. To create a bus in RLF, we use the
[`rlf.Bus`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Bus.html) class and pass
the constructor an identifier and the number of phases that the bus has.

The identifier is used internally by the solver to identify network elements and is also used to name
the element when displaying results. Almost all elements in RLF require an id when instantiating.
Also, a bus in RLF can either be a three-phase bus (with or without neutral i.e. `abcn` or `abc`)
or any combination of one/two phases with or without the neutral. More information on buses and
possible phase combinations in RLF can be found
[here](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Bus.html).

For this tutorial, the network's buses are created as follows:

- `source_bus = rlf.Bus(id='SourceBus', phases='abc', nominal_voltage=20e3)`: creates a 3-phase bus
  with no neutral since it's typically not needed in a medium voltage network.
- `bus_a = rlf.Bus(id="Bus A", phases="abcn", nominal_voltage=400)`: creates bus _A_ as a 3-phase bus
  with a neutral (since it's a distribution network bus). Bus _B_ is created in a similar manner.
- `bus_c = rlf.Bus(id="Bus C", phases="an", nominal_voltage=400)`: creates bus _C_ as a single phase
  bus with a neutral. Buses _D_ and _E_ are created in a similar manner but with different phases.

Note that the `nominal_voltage` parameter is optional but recommended as it allows us to get the
per-unit voltage level automatically when we run the power flow.


In [ ]:
source_bus = rlf.Bus(id="SourceBus", phases="abc", nominal_voltage=20e3)
bus_a = rlf.Bus(id="Bus A", phases="abcn", nominal_voltage=400)
bus_b = rlf.Bus(id="Bus B", phases="abcn", nominal_voltage=400)
bus_c = rlf.Bus(id="Bus C", phases="an", nominal_voltage=400)
bus_d = rlf.Bus(id="Bus D", phases="bn", nominal_voltage=400)
bus_e = rlf.Bus(id="Bus E", phases="cn", nominal_voltage=400)

### Adding a voltage source to the source bus

Because the source bus is responsible for powering the network, we need to add a voltage source at
the bus to reflect this. To add a voltage source to a bus in RLF, we first need to set its voltage
value.

A voltage source can either be $\Delta$ or Y-connected as described
[here](https://roseau-load-flow.roseautechnologies.com/en/stable/models/VoltageSource.html#connections).
Since the source bus has no neutral, therefore, we can only add a $\Delta$-connected voltage source
to it. The complete procedure is as follows:

Create a voltage source using the
[`rlf.VoltageSource`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/VoltageSource.html)
class and specify an id, the bus it is connected to and its voltages:
`vs = VoltageSource(id="Source", bus=source_bus, voltages=20e3)`.

`voltages=20e3` creates a complex array of three-phase voltages with a magnitude of 20 kV and a phase
shift of -120° and 120° between them. Has the bus had a neutral, its voltages would have been defined
as `voltages=20e3 / np.sqrt(3)` to keep the line-to-line voltage at 20 kV.

Note that it is also possible to create an unbalanced voltage source by specifying the complex
voltages of each phase as an array. For example `voltages=[20e3 + 0j, 20e3 + 0j, 20e3 + 0j]` creates
a zero-sequence voltage source.


In [ ]:
vs = rlf.VoltageSource(id="Source", bus=source_bus, voltages=20e3)

### Adding the transformer

Next, we will add the MV/LV transformer. To add a transformer in RLF, first we need to define the
transformer's parameters using the
[`rlf.TransformerParameters`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Transformer/index.html)
class. This can be done manually, using predefined transformers in the catalogue, or using helper
conversion functions from other software. More information is available
[here](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Transformer/index.html).

For the purpose of this tutorial, we will be using predefined transformer parameters from the
catalogue of transformers included in RLF. To do this, we use the `from_catalogue` method of the
`rlf.TransformerParameters` class. In this case, we use the parameters of a 250 kVA Schneider Electric
Minera transformer with the highest efficiency (AA0Ak). More information on using the transformers
catalogue is available
[here](https://roseau-load-flow.roseautechnologies.com/en/stable/usage/Catalogues.html#transformers).


In [ ]:
tp = rlf.TransformerParameters.from_catalogue("SE Minera AA0Ak 250kVA 20kV 410V Dyn11")

Once the transformer's parameters have been defined, a transformer can then be created using the
[`rlf.Transformer`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Transformer/index.html)
class. We pass in the id of the transformer as well as the buses and phases on its primary side (1)
and secondary side (2). Finally, we pass in the transformer's parameters created above.


In [ ]:
transformer = rlf.Transformer(id="Transformer", bus1=source_bus, bus2=bus_a, parameters=tp)

### Adding potential references and grounds

To run power flow calculations in _RLF_, a potential reference must be added to an element in each
galvanically isolated section of the network. For this tutorial, the isolated sections are the
primary and secondary sides of the transformer.

For the primary side of the transformer, we use a
[`rlf.PotentialRef`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/PotentialRef.html)
connected to the `source_bus`. We pass in its id, the target `element` (which can be a `Bus` or a
`Ground`) whose potential will be set to 0 V and an optional `phase` argument which is set to `None`
by default. In this case, as the target bus `source_bus` has three potentials (`a`, `b` and `c`) and
as the `phases` argument is not defined, the sum of these potentials is set to 0 V by `pref_mv`.


In [ ]:
pref_mv = rlf.PotentialRef(id="PRef MV", element=source_bus)

For the LV part of the network, we will be using lines with shunt connections. These lines require a
[`rlf.Ground`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Ground.html) element.
The constructor of this element only takes an id.


In [ ]:
ground = rlf.Ground(id="Ground")

Now, we can connect this ground to the neutral of the secondary side of the transformer using its
`connect` method.


In [ ]:
ground.connect(bus=bus_a, phase="n")

For now, the potential of the element `ground` has not been set to 0 V. In RLF, the ability to define
`Ground` elements has been separated from the ability to set potentials to zero. This is a unique
feature of _RLF_ and will become more familiar with subsequent tutorials.

We then set the potential of the element `ground` to 0 V as the reference for LV potentials using
another `rlf.PotentialRef`.


In [ ]:
pref_lv = rlf.PotentialRef(id="PRef LV", element=ground)

Here, the element provided to the constructor is a `Ground` element. As a `Ground` has a single
potential, the `phase` argument of the constructor must be ignored.


### Adding Lines

Next, we'll add in all the lines present in the network. Similar to the transformer, to add lines in
RLF, we first need to specify the parameters for the line and this can be done manually using the
[`rlf.LineParameters`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Line/Parameters.html)
or with the catalogue. For manual entry, we will need to specify the series impedance matrix `z_line`
and optionally, the shunt admittance matrix `y_shunt`. The dimensions of these matrices must be equal
to the number of conductors (phases) of the line.

For simplicity, we will use the RLF catalogue to get predefined parameters for lines. To do this we
pass in the name of the desired line parameter as well as the number of phases required which can
range from 1 to 4. For the three-phase line (with neutral) between buses _A_ and _B_, we will use
parameters from an overhead aluminium cable with a cross-sectional area of 240 mm². The single-phase
lines will be modelled using parameters from an underground aluminium cable with a cross-sectional
area of 19 mm² as shown below. Using line parameters make it easy to create multiple lines with the
same characteristics.

- `lp_240 = rlf.LineParameters.from_catalogue("O_AL_240", nb_phases=4)`
- `lp_19 = rlf.LineParameters.from_catalogue("U_AL_19", nb_phases=2)`

Next, we create the actual lines using the
[`rlf.Line`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Line/index.html) class.
To do this, we pass the constructor the line's id, the buses it's connected to (bus 1 and bus 2), the
length of the line, and the line's parameters. In RLF, when the `y_shunt` matrix is defined in the
line parameters, the line requires a ground for shunt connections. Here, we reuse the same ground
element for the LV part of the network.

- `line_ab = rlf.Line(id="Line A-B", bus1=bus_a, bus2=bus_b, parameters=lp_240, phases="abcn", length=rlf.Q_(1.0, "km"), ground=ground)`

**Note**: To demonstrate the flexibility of RLF, the length of the line may be specified in any unit
unit of distance such as kilometer, miles, meters, etc using the `rlf.Q_` (Quantity) class. This class
takes in the numerical value of the length and its unit.

However, it should also be noted that most arguments in RLF functions, methods or classes can also be
given without using the `rlf.Q_` constructor. In this case the default unit used by that function,
class or method will be assumed.


In [ ]:
# Getting line parameters from the catalogue
lp_240 = rlf.LineParameters.from_catalogue("O_AL_240", nb_phases=4)
lp_19 = rlf.LineParameters.from_catalogue("U_AL_19", nb_phases=2)

# Creating the actual lines
line_ab = rlf.Line(
    id="Line A-B", bus1=bus_a, bus2=bus_b, parameters=lp_240, phases="abcn", length=rlf.Q_(1.0, "km"), ground=ground
)
line_bc = rlf.Line(
    id="Line B-C", bus1=bus_b, bus2=bus_c, phases="an", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)
line_bd = rlf.Line(
    id="Line B-D", bus1=bus_b, bus2=bus_d, phases="bn", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)
line_be = rlf.Line(
    id="Line B-E", bus1=bus_b, bus2=bus_e, phases="cn", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)

### Adding loads

In the final modelling step, we will add the loads at buses C, D and E. The description of the loads
are given in the table below. RLF can model different types of loads as described
[here](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Load/index.html).

| Load Name | Phases | Connected bus | Peak Demand (kW) | PF   |
| :-------- | :----- | :------------ | :--------------- | :--- |
| Load 1    | an     | C             | 7                | 0.95 |
| Load 2    | bn     | D             | 6                | 0.95 |
| Load 3    | cn     | E             | 8                | 0.95 |

For this tutorial, the loads are assumed to be drawing constant power irrespective of the voltage.
This means we can model them using a
[`rlf.PowerLoad`](https://roseau-load-flow.roseautechnologies.com/en/stable/models/Load/PowerLoad.html)
class. This class takes in the id of the load, the bus it is connected to, the phases of that bus it
is attached to, and the complex apparent power of the load. The power must be given as an array of
one or three values for single and three-phase loads respectively.

Since we are given active power and power factor instead of the complex apparent power, we need to
carry out a conversion. For this, w write a small helper function, `convert_power` that takes in the
peak active power demand and the power factor and spits out the complex power as follows:

$\underline{S} = P \left[1 + j\tan\left(\cos^{-1}(pf)\right)\right]$


In [ ]:
def convert_power(power: float, pf: float) -> complex:
    """Convert active power and power factor to complex power."""
    return power * (1 + np.tan(np.arccos(pf)) * 1j)

Then, we can finally create the load as follows:


In [ ]:
load1 = rlf.PowerLoad(id="Load 1", bus=bus_c, phases="an", powers=[convert_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="Load 2", bus=bus_d, phases="bn", powers=[convert_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="Load 3", bus=bus_e, phases="cn", powers=[convert_power(8e3, 0.95)])

Note that these three loads are connected between a phase and the neutral of their buses that has the
exact same phases as the loads. When the phases of the bus and the load are the same, specifying the
phases of the load is redundant and can be skipped.


### Building the network

Following the modelling of all network elements, the next step is to build the actual network.
Currently, elements exist in isolation and they do not belong to any network. This can be verified
by accessing the `network` attribute of an element.


In [ ]:
load1.network is None

To build the network in _RLF_, we use the `rlf.ElectricalNetwork` class. We can either pass all the
elements to the class constructor manually of use the `from_element` method to build the entire
network from one of the buses instead. Here, we pass in the source bus to `from_element` then print
the network to see what it contains.


In [ ]:
en = rlf.ElectricalNetwork.from_element(source_bus)
en

We can see that the network contains all the six buses, the four lines, the transformer, the three
loads, the voltage source, the ground and the two potential references. The network is now ready to
be used for power flow calculations.


### Running a load flow

To execute load flow calculations in _RLF_, a license is required. A free license limited to networks
with at most 10 buses key is available
[`here`](https://roseau-load-flow.roseautechnologies.com/en/stable/License.html#license-types) and
can be used for the purpose of this tutorial.

Visit the link above to see how to activate the license. Note also that students and researchers can
request a free academic license that is not limited in the number of buses. More information on this
can be found in the link above.

Once the license is properly activated, we can simply run a load flow as shown below:


In [ ]:
en.solve_load_flow()

This function returns the number of iterations performed by the solver and the residual error after convergence.


### Accessing results

After running the load flow calculations, we can extract the value of several quantities from
different network elements. Each quantity can be accessed through the name of the quantity prefixed
with `res_`. A complete breakdown of available results for each element is given in the table below:

| Element type                                | Available results                                                                                                                                                      |
| ------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `Bus`                                       | `res_potentials`, `res_voltages`, `res_voltage_levels`, `res_violated`                                                                                                 |
| `Line`                                      | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_series_power_losses`, `res_shunt_power_losses`, `res_power_losses`, `res_loading`, `res_violated` |
| `Transformer`                               | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_power_losses`, `res_loading`, `res_violated`                                                      |
| `Switch`                                    | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                                         |
| `ImpedanceLoad`, `CurrentLoad`, `PowerLoad` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_flexible_powers`&#8270;                                                                           |
| `VoltageSource`                             | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                                         |
| `Ground`                                    | `res_potential`                                                                                                                                                        |
| `PotentialRef`                              | `res_current` _(Always zero for a successful load flow)_                                                                                                               |

&#8270;: `res_flexible_powers` is only available for flexible loads (`PowerLoad`s with `flexible_params`).

To access results for different elements in _RLF_, there are two main methods. The first method
involves calling the `res_` method for the desired quantity for that element e.g.


In [ ]:
load1.res_voltages

Note that the return type is a Quantity (`rlf.Q_`) class. To get the numerical value of the quantity,
we can use its `.m` property (`m` stands for magnitude).


In [ ]:
load1.res_voltages.m

However, this method of accessing results can only be used for one element at a time. To extract
results for all elements of the same type, we can call the `res_` method of the electrical network
instead. For example, to get the results for all the loads in the network, we can use `en.res_loads`
as shown below.


In [ ]:
en.res_loads

We get a `DataFrame` (from the _Pandas_ library) of results. In a data frame, the units are not
included thus the values are always displayed in SI units of V, A and W.


##### Accessing active and reactive power of loads

Although, these were explicitly specified, we can still verify the results. We will apply both
methods described above to retrieve the active (P) and reactive (Q) powers of the three loads present
in the network. Note the `res_power` method in RLF returns the powers flowing in the live and neutral
conductors. Therefore, the total power dissipated by the load can be determined by taking a sum of
these values as shown below.


In [ ]:
# Method 1
load_power = load1.res_powers.m_as("kVA").sum()
print(f"Active power of Load 1: {round(load_power.real, 3)} kW")
print(f"Reactive power of Load 1: {round(load_power.imag, 3)} kVAr")

Here, the result is converted to kVA using the `m_as` method of the quantities (`Q_`) class. This
method returns the magnitude of the quantity in the desired unit.


In [ ]:
# Method 2
# We extract the power column from the DataFrame of results
load_powers = en.res_loads["power"].sort_index() / 1e3

# We extract the total active and reactive power of each load
for load_id in load_powers.index.levels[0]:
    s = load_powers.loc[load_id].sum()
    print(f"{load_id}: P = {s.real:.3f} kW, Q = {s.imag:.3f} kVAr")

##### Voltages of the loads and the three-phase buses

Similar to the loads, we can extract the voltage of a certain bus using its `res_voltages` property.
This returns the voltage in complex form which can be converted into polar form to extract the
magnitude and phase angle.

To extract the results for multiple or all buses simultaneously, we can use the `res_buses_voltages`
property of the electrical network. This returns a dataframe which can be filtered to get the results
of the desired buses. By default, the results are also in rectangular format. To convert to polar
form, we use the `transform` method of dataframes.

Both methods are shown below.


In [ ]:
# Method 1
# Voltage result at bus C
vc = bus_c.res_voltages.m[0]
# Get the magnitude and angle of the voltage at bus C
print(f"|Vc| = {np.abs(vc):.3f} V, angle(Vc) = {np.angle(vc, deg=True):.3f}°")

In [ ]:
# Method 2
# Voltage result for all buses
v_buses = en.res_buses_voltages["voltage"]
# Transform the complex voltage to magnitude and angle
v_buses.transform({"magnitude": abs, "angle °": lambda v: np.angle(v, deg=True)})

Additionally, since we defined the optional `nominal_voltage` parameter for the buses. We can extract
the voltage levels of the buses using the `res_voltage_levels` property. This is the per-unit voltage
of the buses with respect to their nominal voltage.


In [ ]:
# Method 1
# Voltage level at bus C in per unit
bus_c.res_voltage_levels.m

In [ ]:
# Method 2
# Voltage level at all buses
en.res_buses_voltages  # ["voltage_level"]

#### Active and Reactive powers of the transformer

Similarly, to extract the active and reactive powers for the transformer, we can use the two methods
of accessing results as shown below. It can be observed that the powers in the secondary side of the
transformer are negative. This indicates that the power is leaving the transformer as opposed to the
positive powers on the primary side entering the transformer. Also, there is no power flowing
thorough the neutral phase as expected.


In [ ]:
# Method 1
# Primary and secondary complex powers of a single transformer (in kVA and rounded)
s_primary, s_secondary = transformer.res_powers
np.round(s_primary.m_as("kVA"), 3), np.round(s_secondary.m_as("kVA"), 3)

In [ ]:
# Method 2
# Primary and secondary complex powers of all transformers (in kVA and rounded)
(en.res_transformers[["power1", "power2"]] / 1e3).round(3)

In addition, we can also retrieve the power lost in the transformer using the `res_power_losses`
property of the `rlf.Transformer` object as shown below.


In [ ]:
transformer.res_power_losses.m

#### Line Losses

Finally, the line losses on the network can also be conveniently accessed using the described methods.
The first method uses the `res_series_power_losses` of each line and returns the line losses for each
phase. The second method uses the `res_lines` property of the electrical network and the column
`series_losses` in the resulting dataframe contains the power losses on each phase. Conventionally,
when talking about line losses, we are typically interested in the active power losses in the lines
which can be obtained as shown below.


In [ ]:
# Method 1
# Series Watt losses of a single line (unit: W)
np.real(line_ab.res_series_power_losses.m)

In [ ]:
# Method 2
# Series Watt losses of all lines (unit: W)
en.res_lines["series_losses"].transform({"active_power_losses": np.real})